## Loading 3D pose output files
This short script demonstrates how to load and visualise the generated datasets.

Each dataset consists of 3 file types:
* **n** images
* **1** 3D pose dataframe
* **1** label lookup table, containing keypoint names

The **.hdf5** formatted file dataframe contains the following entries for each generated image:

*	  file_name &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1 string (relative)
*	  rot_mat	&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3 x 3 float
*	  trans_mat	&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3 x 1 float
*	  intrinsics_mat &nbsp;&nbsp;&nbsp; 3 x 3 float
*     bounding_box &nbsp;&nbsp;&nbsp; 4 x 1 float
*	  key_points_3D &nbsp;&nbsp;&nbsp;3 x k float (provide name sheet)
*	  key_points_2D &nbsp;&nbsp;&nbsp;2 x k float
*	  visibility &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1 x k int (0 occluded or 1 visible)

In [ ]:
import os
import cv2

import numpy as np
import pandas as pd

target_dir = "../data/single_sungaya/"

out_df = pd.read_hdf("../data/single_sungaya/Data_3D_Pose.hdf5")
out_df

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

### Load and display 3D coordinates
Change **show_entry** to the id of the sample, you want to examine

Check the dataframe above to see which id corresponds to which image.

In [ ]:
def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    if len(points) < 62:
        return
    elif len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb])
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4)
        return ax

def centralise_3d(sample):
        x_diff, y_diff, z_diff = sample[3-1][0], sample[3-1][1], sample[3-1][2]
        for i in range(len(sample)):
            sample[i][0] = sample[i][0] - x_diff
            sample[i][1] = sample[i][1] - y_diff
            sample[i][2] = sample[i][2] - z_diff
        return sample

In [ ]:
#  Checks the visibility of each keypoint determining that the most meaningful keypoint that is on the main body of the bug is the 3rd
z= np.zeros((62))
for col, x in out_df.iterrows():
    z += np.array(x['visibility'])
print(np.argmax(z))
z

In [ ]:
filefind = "7744_Img_synth.png"
out_df.loc[out_df['file_name'] == filefind]

In [ ]:
show_entry = 2863
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')

display_points_3D = out_df.loc[show_entry]["key_points_3D"]
visibility_mask = out_df.loc[show_entry]["visibility"]
display_img = cv2.imread(os.path.join(target_dir, out_df.loc[show_entry]["file_name"]))

# points = np.array(centralise_3d(display_points_3D))

plot_stick_bug(ax, display_points_3D, visibility_mask)
# for i,xyz in enumerate(display_points_3D):
#     if out_df.loc[show_entry]["visibility"][i] == 1:
#         if i== 6:
#             ax.scatter(xyz[0], xyz[1], xyz[2]-2.5, marker='o',s=10)

# """
# # additionally, plot the camera location
# ax.scatter(out_df.loc[show_entry]["cam_trans"][0], 
#            out_df.loc[show_entry]["cam_trans"][1], 
#            out_df.loc[show_entry]["cam_trans"][2], marker='x')
# """

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)

# # opens external plot
plt.title(out_df.loc[show_entry]["file_name"])
plt.show()

### Load and display 2D coordinates 
as well as **3D coordinates** projected onto the **camera view**.

In [ ]:
# file_name	cam_rot	cam_trans	cam_intrinsics	bounding_box	key_points_3D	key_points_2D	visibility
R = np.array(out_df.loc[show_entry]["cam_rot"])
T = np.reshape(np.array(out_df.loc[show_entry]["cam_trans"]),(3,1))
C = np.array(out_df.loc[show_entry]["cam_intrinsics"])
bbox = np.array(out_df.loc[show_entry]["bounding_box"])

fig = plt.figure()
ax = fig.add_subplot()

# display the generated image
ax.imshow(display_img)
rect=plt.Rectangle((bbox[0],bbox[1]),bbox[2]-bbox[0],bbox[3]-bbox[1], 
                        fill = False,
                        color = "purple",
                        linewidth = 2)
    
for i, x in enumerate(display_points_3D):
    X = np.reshape(np.array(out_df.loc[show_entry]["key_points_3D"][i]),(3,-1))

#     # given the above data, it should be possible to project the 3D points into the corresponding image,
#     # so they land in the correct position on the image 
    P = C @ np.hstack([R, T])  # projection matrix
    X_hom = np.vstack([X, np.ones(X.shape[1])])  # 3D points in homogenous coordinates
    # print(X_hom)
    X_hom = P @ X_hom  # project the 3D points
    # print(X_hom)
    X_2d = X_hom[:2, :] / X_hom[2, :]  # convert them back to 2D pixel space
    # print(X_2d)
    gt_x_2d = out_df.loc[show_entry]["key_points_2D"][i][0]
    gt_y_2d = out_df.loc[show_entry]["key_points_2D"][i][1]
    
    ax.scatter(gt_x_2d, gt_y_2d, marker='o', s=10,  color='b')
    ax.scatter(X_2d[0], display_img.shape[1]-X_2d[1], marker='^',  color='r',s=2) #3d projected points

# Displays specific joints
# i, o = 3, 3
# for i in range(i-1,o):
#     gt_x_2d = out_df.loc[show_entry]["key_points_2D"][i][0]
#     gt_y_2d = out_df.loc[show_entry]["key_points_2D"][i][1]
#     ax.scatter(gt_x_2d, gt_y_2d, marker='o', s=10)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')

ax.set_xlim([0,display_img.shape[0]])
ax.set_ylim([0,display_img.shape[1]])
ax.set_aspect('equal')

plt.gca().add_patch(rect)

plt.title(out_df.loc[show_entry]["file_name"] + "_projected")
plt.show()